In [19]:
import chromadb
from chromadb.config import Settings
from os import getcwd, environ
from pathlib import Path
from sys import path
import uuid
from random import randint
path.append(r'C:\Users\Izogie\Desktop\Folders\Projects\Python\KB Chat\src')
from modules.SourceManager import SourceManager

In [2]:
PROJ_DIR = Path(getcwd()).parent
DB_DIR = PROJ_DIR / "chroma"

In [3]:
client = chromadb.PersistentClient(path=str(DB_DIR), settings=Settings(allow_reset=True))

c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


In [4]:
def fresh_start(client):
    client.reset()
    collection = client.get_or_create_collection("coppermind")
    return collection

In [5]:
manager = SourceManager()
data = manager.load_json("processed_articles.jsonl")

In [6]:
import pandas as pd
df = pd.DataFrame(data)
df

C:\Users\Izogie\AppData\Local\Temp\ipykernel_25312\1927957191.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,title,links,sections
0,Kaladin,"[Stormfather, Cognitive Shadow, Dalinar Kholin...","[{'title': 'Kaladin', 'content': 'Kaladin, nic..."
1,Pits of Hathsin,"[Cosmere, Kelsier]","[{'title': 'Pits of Hathsin', 'content': 'The ..."
2,Allomancy,"[Odium, Kelsier, Realmatic Theory, Hoid, Knigh...","[{'title': 'Allomancy', 'content': 'Allomancy ..."
3,Cosmere,"[Stormfather, Kelsier, Cognitive Shadow, Dalin...","[{'title': 'Cosmere', 'content': 'The cosmere ..."
4,Honor's Perpendicularity,"[Odium, Stormfather, Shadesmar, Surgebinder, F...","[{'title': 'Honor's Perpendicularity', 'conten..."
5,Cephandrius,[Hoid],None
6,Hoid,"[Kelsier, Cognitive Shadow, Dalinar Kholin, Su...","[{'title': 'Hoid', 'content': 'Hoid is a world..."
7,Odium,"[Honor's Perpendicularity, Stormfather, Hoid, ...","[{'title': 'Odium', 'content': '<disable-googl..."
8,Realmatic Theory,"[Cosmere, Honor's Perpendicularity, Stormfathe...","[{'title': 'Realmatic Theory', 'content': 'Rea..."
9,Knights Radiant,"[Cosmere, Stormfather, Hoid, Dalinar Kholin, K...","[{'title': 'Knights Radiant', 'content': 'The ..."


In [42]:
def ingest_articles(collection, articles):
    """Ingests structured article data into ChromaDB."""
    namespace_uuid = uuid.UUID('f81d4fae-7dec-11d0-a765-00a0c91e6bf6')
    documents = []
    metadatas = []
    ids = []

    for article in articles:
        if article["sections"] is None:
            documents.append("")
            metadatas.append({
                "article_title": article["title"],
                "paragraph_header": paragraph['title'],
                "paragraph_order": 0,
                "links": ', '.join(article["links"]),
            })
            ids.append(str(uuid.uuid5(namespace_uuid, article['title'])))
        else:
            for paragraph in article["sections"]:
                documents.append(paragraph["content"])
                metadatas.append({
                    "article_title": article["title"],
                    "paragraph_header": paragraph['title'],
                    "paragraph_order": paragraph["order"],
                    "links": ', '.join(article["links"]),
                })
                ids.append(str(uuid.uuid5(namespace_uuid, f"{paragraph['title']}_{paragraph['order']}_{randint(0,10000)}")))  # Create unique IDs

    collection.add(
        documents=documents,
        metadatas=metadatas,
        ids=ids 
    )
    return collection

In [95]:
sents_collection = fresh_start(client)
sents_collection = ingest_articles(sents_collection, data)  

c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\reque

In [71]:
sents_collection = client.get_or_create_collection("coppermind")

c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


In [92]:
query = "what does steel do in allomancy"
sents_response = sents_collection.query(query_texts=query, n_results=10, include=["documents", "metadatas","distances"])


In [93]:
for idx, meta in enumerate(sents_response['metadatas'][0]):
    print(f"{meta['article_title']}, {meta['paragraph_header']}")
    print(sents_response["documents"][0][idx] + f"\n {open_response["distances"][0][idx]}")

Allomancy, Allomantic Metals
There are sixteen basic Allomantic metals. The metals come in pairs: one metal is a periodic element, while the other is the alloy of that base metal. These pairs have complementary effects. Iron Pulls on nearby metals, while its alloy, steel, Pushes on metals. Similarly, each elemental metal is classified as a "Pulling" ability, whereas each alloy is classified as a "Pushing" ability. The purity of the metal corresponds to its effectiveness and alloys require precise compound percentages for maximum power. Burning an impure alloy may cause sickness, while metals that are not Allomantically viable cannot be burned after being ingested.
 0.3441183865070343
Allomancy, Mechanics
Allomantic metals are the focus, but not the source, of Allomancy's power. When an Allomancer burns a metal, that metal's specific molecular structure acts as a conduit to Preservation, and then the Allomancer is granted an ability corresponding to the metal, allowing the metal to "foc

# OpenAI embeddings

In [40]:
import chromadb.utils.embedding_functions as embedding_fns

In [39]:
# Load API key from config.yaml (`GOOGLE_API_KEY: <YOUR API TOKEN>`) into environment variable.
# Kept in a seperate file for ease of access across projects.
from yaml import safe_load
def load_config(file_path="./config.yml"):
    with open(file_path, 'r') as file:
        config = safe_load(file)
        for key, value in config.items():
            environ[key] = value
load_config()

In [74]:
def ingest_articles(collection, articles):
    """Ingests structured article data into ChromaDB."""
    namespace_uuid = uuid.UUID('f81d4fae-7dec-11d0-a765-00a0c91e6bf6')
    documents = []
    metadatas = []
    ids = []

    for article in articles:
        if article["sections"] is None:
            continue
        else:
            for paragraph in article["sections"]:
                documents.append(paragraph["content"])
                metadatas.append({
                    "article_title": article["title"],
                    "paragraph_header": paragraph['title'],
                    "paragraph_order": paragraph["order"],
                    "links": ', '.join(article["links"]),
                })
                ids.append(str(uuid.uuid5(namespace_uuid, f"{paragraph['title']}_{paragraph['order']}_{randint(0,10000)}")))  # Create unique IDs

    collection.add(
        documents=documents,
        metadatas=metadatas,
        ids=ids 
    )
    return collection

In [75]:
openai_ef = embedding_fns.OpenAIEmbeddingFunction(api_key=environ["OPENAI_API_KEY"])

In [76]:
# client.reset()
open_collection = client.get_or_create_collection("open_coppermind", embedding_function=openai_ef)
open_collection = ingest_articles(open_collection, data)

c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


In [88]:
query = "what does steel do in allomancy"
open_response = open_collection.query(query_texts=query, n_results=10, include=["documents", "metadatas","distances"])


In [90]:
for idx, meta in enumerate(open_response['metadatas'][0]):
    print(f"{meta['article_title']}, {meta['paragraph_header']}")
    print(open_response["documents"][0][idx] + f"\n {open_response["distances"][0][idx]}" )

Allomancy, Mechanics
Allomantic metals are the focus, but not the source, of Allomancy's power. When an Allomancer burns a metal, that metal's specific molecular structure acts as a conduit to Preservation, and then the Allomancer is granted an ability corresponding to the metal, allowing the metal to "focus" Preservation's power to the user. Each metal is the only way a mortal can access Preservation and the power of creation, after which it is vaporized. However, this is only true for the base sixteen Allomantic metals and not God Metals or their alloys. 
 0.3441183865070343
Allomancy, Mechanics
Allomancy is an End-Positive magic system, which draws on the power of Preservation, channeled through a metal when it is burned. This differs to how Feruchemy works, as Allomancy is not being powered by the body of the Allomancer, but instead by an external force. Like the other Metallic Arts, Allomancy as a magic system formed through the natural interactions between the two Shards that inh

# Google Gemini embedding

In [48]:
import chromadb.utils.embedding_functions as embedding_fns
import google.generativeai as genai

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [51]:
genai.configure(api_key=environ['GOOGLE_API_KEY'])
embedding_function = embedding_fns.GooglePalmEmbeddingFunction(api_key=environ["GOOGLE_API_KEY"])

In [80]:
# client.reset()
goog_collection = client.get_or_create_collection("goog_coppermind", embedding_function=embedding_function)
goog_collection = ingest_articles(goog_collection, data)

c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


In [83]:
query = "what does steel do in allomancy"
goog_response = goog_collection.query(query_texts=query, n_results=10, include=["documents", "metadatas","distances"])


In [91]:
for idx, meta in enumerate(goog_response['metadatas'][0]):
    print(f"{meta['article_title']}, {meta['paragraph_header']}")
    print(goog_response["documents"][0][idx] + f"\n {open_response["distances"][0][idx]}")

Allomancy, Allomantic Metals
The Allomantic effects of metals are most similar to the effects of fabrials on Roshar. Steel and Iron also have corresponding effects in how they affect Aether, whether other allomantic metals also have effects or not is currently unknown.
 0.3441183865070343
Allomancy, Mechanics
Allomancy is an End-Positive magic system, which draws on the power of Preservation, channeled through a metal when it is burned. This differs to how Feruchemy works, as Allomancy is not being powered by the body of the Allomancer, but instead by an external force. Like the other Metallic Arts, Allomancy as a magic system formed through the natural interactions between the two Shards that inhabit Scadrial, Preservation and Ruin, and the planet itself, instead of being created by a specific Shard. This magic system, however, is generally tailored and fuelled by Preservation, although specific cases will involve an Allomancer drawing upon Ruin's essence instead. Like other forms of 